# Binary Logistic Regression

## Preparing the data

In [13]:
eel <- read.delim("data/eel.dat",header=T)

head(eel)

,Cured,Intervention,Duration
,<chr>,<chr>,<int>
1,Not Cured,No Treatment,7
2,Not Cured,No Treatment,7
3,Not Cured,No Treatment,6
4,Cured,No Treatment,8
5,Cured,Intervention,7
6,Cured,No Treatment,6


## Create Categories

In [14]:
eel$Cured = factor(eel$Cured)
eel$Intervention = factor(eel$Intervention)

summary(eel)

       Cured          Intervention    Duration    
 Cured    :65   Intervention:57    Min.   : 4.00  
 Not Cured:48   No Treatment:56    1st Qu.: 6.00  
                                   Median : 7.00  
                                   Mean   : 7.08  
                                   3rd Qu.: 8.00  
                                   Max.   :10.00  

### Reorder the categories

Change the order of factors in **Cured** and **Intervention**.<br>
Now "Not Cured" is the first factor of **Cured** , and so on.

In [15]:
eel$Cured <- relevel(eel$Cured, "Not Cured")
eel$Intervention <- relevel(eel$Intervention, "No Treatment")

summary(eel)

       Cured          Intervention    Duration    
 Not Cured:48   No Treatment:56    Min.   : 4.00  
 Cured    :65   Intervention:57    1st Qu.: 6.00  
                                   Median : 7.00  
                                   Mean   : 7.08  
                                   3rd Qu.: 8.00  
                                   Max.   :10.00  

## Basic Logistic Regression analysis

Use **Intervention** as the only predictor.

In [16]:
eelModel.1 <- glm(Cured ~ Intervention, data = eel, family = binomial())

summary(eelModel.1)


Call:
glm(formula = Cured ~ Intervention, family = binomial(), data = eel)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5940  -1.0579   0.8118   0.8118   1.3018  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)   
(Intercept)               -0.2877     0.2700  -1.065  0.28671   
InterventionIntervention   1.2287     0.3998   3.074  0.00212 **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 154.08  on 112  degrees of freedom
Residual deviance: 144.16  on 111  degrees of freedom
AIC: 148.16

Number of Fisher Scoring iterations: 4


Use **Intervention** and **Duration** as the predicators.

### Calculate the **model chi-square statistics**

$$\chi^2=(-2LL_{new}) - (-2LL_{baseline})$$
$$df = df_{new} - df_{baseline}$$

In [17]:
modelChi <- eelModel.1$null.deviance - eelModel.1$deviance

modelDf <- eelModel.1$df.null - eelModel.1$df.residual

chisq.prob <- 1 - pchisq(modelChi, modelDf)

chisq.prob

[1] 0.001629425

**chi-square statistics** < 0.0.5 , so the model is **significantly** better than predicting the outcome by the chance.

### Calcuate R-statistic

The **R-statistic** is the partial correlation between the outcome variable and each of the predictor variables.

$$R=\sqrt{\frac{z^2-2df}{-2LL(baseline)}}$$

* $-2LL(baseline)$ is the deviance of the baseline model (null deviance, $154.08$)
* $z$ is the **Wald statistic** (z value of the added predicators, $3.074$) 
* $df$ is the **added** degree of freedom ($1=112-111$)

$R=\sqrt{\frac{3.074^2-2*1}{154.08}}$

In [28]:
z=eelModel.1$effect[2]

modelR = sqrt((z^2-2*modelDf)/(eelModel.1$null.deviance))
modelR

InterventionIntervention 
               0.2198356

Note: It's **invalid** to square this R-stastics value and interpret it as you would in linear egessions!

### Other analogues to the $R^2$ in linear regression

Hosmer and Lemeshow's ($R_L^2$) measure:

$$R_L^2=\frac{-2LL_{model}}{-2LL_{baseline}}=\frac{(-2LL_{baseline})-(-2LL_{new})}{-2LL_{baseline}}$$

In [37]:
R2.hl = modelChi / eelModel.1$null.deviance
R2.hl

[1] 0.06442071

Cox's and Shell's $R_{CS}^2$ :

$$R_CS^2=1-\huge{e}^{\frac{(-2LL_{new})-(-2LL_{baseline})}{n}}$$

In [47]:
n=nrow(eel)
n

[1] 113

In [48]:
R2.cs  = 1-exp((eelModel.1$deviance-eelModel.1$null.deviance)/n)
R2.cs

[1] 0.08409487

## The Second Model

In [50]:
eelModel.2 <- glm(Cured ~ Intervention + Duration, data=eel, family = binomial())

summary(eelModel.2)


Call:
glm(formula = Cured ~ Intervention + Duration, family = binomial(), 
    data = eel)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6025  -1.0572   0.8107   0.8161   1.3095  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)              -0.234660   1.220563  -0.192  0.84754   
InterventionIntervention  1.233532   0.414565   2.975  0.00293 **
Duration                 -0.007835   0.175913  -0.045  0.96447   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 154.08  on 112  degrees of freedom
Residual deviance: 144.16  on 110  degrees of freedom
AIC: 150.16

Number of Fisher Scoring iterations: 4


### Comparing model 2 and model 1

#### Using AIC

the AIC of model 1(148.16) and model 2(150.16), Model 1's AIC is lower,so it's better.

#### Using the chi-square statistic

In [54]:
modelChi <- eelModel.1$deviance - eelModel.2$deviance
chidf <- eelModel.1$df.residual - eelModel.2$df.residual 
chisq.prob <- 1 - pchisq(modelChi, chidf)
chisq.prob

[1] 0.9644765

p-value is greater than 0.05, so Model 2 is not significantly better.

We can calcuale the chi-square deviance and degree of freedom using anova()

In [57]:
anova(eelModel.1,eelModel.2)

,Resid. Df,Resid. Dev,Df,Deviance
,<dbl>,<dbl>,<dbl>,<dbl>
1,111,144.1578,NA,NA
2,110,144.1558,1,0.001983528


## Casewise diagnostics for Model 1

### Calculate the residual statistics

In [68]:
eel$predicted.probabilities <- fitted(eelModel.1)
eel$standardized.residuals <- rstandard(eelModel.1)
eel$studentized.residuals <- rstudent(eelModel.1)
eel$dfBeta <- dfbeta(eelModel.1)
eel$dffit <- dffits(eelModel.1)
eel$leverage <- hatvalues(eelModel.1)

head(eel[c("predicted.probabilities","standardized.residuals","studentized.residuals","dfBeta","dffit","leverage")])

,predicted.probabilities,standardized.residuals,studentized.residuals,dfBeta,dffit,leverage
,<dbl>,<dbl>,<dbl>,"<dbl[,2]>",<dbl>,<dbl>
1,0.4285714,-1.0675117,-1.0643627,"-3.886912e-02, 0.03886912",-0.12623854,0.01785714
2,0.4285714,-1.0675117,-1.0643627,"-3.886912e-02, 0.03886912",-0.12623854,0.01785714
3,0.4285714,-1.0675117,-1.0643627,"-3.886912e-02, 0.03886912",-0.12623854,0.01785714
4,0.4285714,1.3135473,1.3110447,"4.782751e-02, -0.04782751",0.15565258,0.01785714
5,0.7192982,0.8189783,0.8160435,"2.283164e-18, 0.03225994",0.09582269,0.01754386
6,0.4285714,1.3135473,1.3110447,"4.782751e-02, -0.04782751",0.15565258,0.01785714


**Summary of  residual statistics:**

| Name | Comment|
|------|--------|
| **Leverage** | Lies between 0 (no influence) and 1 (complete influence). The expected leverage is $(k+1)/N$ , where $k$ is the number of predictors and $N$ is the sample size. In this case it wourld be $2/113=0.18$.  |
| **Studentized residual** <br /> **Standardized residual** | Only 5% should lie $\pm 1.96$, and about 1% should lie outside $\pm 2.58$. Case above 3 are cause for concern and cases close  to 3 warrant inspection |
| **DFBeta for the constant** <br /> **DFBeta for the first predictor** | should be less than 1 |

## Calculate the effect size

We use the **odds ratios** to evaluate the effect size. It is an indicator of the change in odds resulting from a unit change in the predictor. Each predictor has an odds ratio value.

In [69]:
exp(eelModel.1$coefficients)

(Intercept) InterventionIntervention 
                0.750000                 3.416667